In [6]:
import sys
import os
from datetime import datetime, timedelta
import requests
from sqlalchemy.orm import Session

# Importez la session de DB et la fonction CRUD/l’utilitaire depuis votre projet
# Ajustez les chemins d'import selon votre structure réelle
from database import SessionLocal
from config import settings
from crud import create_article
from utils.openai import generate_embedding

ValidationError: 4 validation errors for Settings
DATABASE_URL
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
SECRET_KEY
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
NEWS_API_KEY
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
OPENAI_API_KEY
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
def fetch_and_populate_articles(
    db: Session,
    from_date: str,
    to_date: str,
    page_size: int = 100,
    sort_by: str = "popularity",
):
    """
    Récupère les articles de NewsAPI sur une plage de dates spécifique
    et les insère dans la base avec leurs embeddings.
    """
    # Endpoint Everything
    url = "https://newsapi.org/v2/everything"

    headers = {
        "Authorization": f"Bearer {settings.NEWS_API_KEY}"
    }

    # Nous ne faisons qu'une seule page, pageSize=100 (maximum)
    # pour avoir les 100 articles les plus "gros" (selon sortBy).
    params = {
        "from": from_date,
        "to": to_date,
        "language": "en",         # ou autre langue si voulu
        "pageSize": page_size,
        "page": 1,                # on ne pagine pas davantage
        "sortBy": sort_by,
        # Vous pouvez laisser q vide (ou préciser un sujet) :
        # "q": "technology"
    }

    print(f"[INFO] Fetching articles from {from_date} to {to_date} ...")
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f"Erreur {response.status_code} : {response.text}")
        return

    data = response.json()
    articles = data.get("articles", [])
    print(f"[INFO] {len(articles)} articles trouvés pour la date {from_date} -> {to_date}")

    for item in articles:
        title = item.get("title")
        content = item.get("content")
        published_str = item.get("publishedAt")
        url_article = item.get("url")

        if not title or not content or not published_str or not url_article:
            # On ignore les articles incomplets
            continue

        try:
            # Convertir la date en datetime
            published_dt = datetime.strptime(published_str, "%Y-%m-%dT%H:%M:%SZ")
        except ValueError:
            continue

        # Limiter le contenu brut à 1000 caractères, par exemple
        raw_text = content[:1000]

        # Générer l'embedding
        embedding = generate_embedding(raw_text)

        article_data = {
            "title": title,
            "summary": None,  # Résumé non généré pour l'instant
            "raw_text": raw_text,
            "published_at": published_dt,
            "url": url_article,
            "embedding": embedding,
            "subjects": []  # Optionnel
        }

        create_article(db, article_data)

In [ ]:
def main():
    """
    Récupère pour chaque jour des 7 derniers jours, les 100 articles les plus populaires
    et les insère dans la base de données.
    """
    db = SessionLocal()
    today = datetime.utcnow()

    # Pour les 7 derniers jours
    for i in range(7):
        day = today - timedelta(days=i)
        # De minuit à minuit
        from_date = day.strftime("%Y-%m-%dT00:00:00")
        to_date = day.strftime("%Y-%m-%dT23:59:59")

        # On peuple la base pour cette journée
        fetch_and_populate_articles(
            db=db,
            from_date=from_date,
            to_date=to_date,
            page_size=100,
            sort_by="popularity"  # 'popularity' ou 'publishedAt'
        )

    db.close()
    print("[INFO] Base de données peuplée avec succès pour les 7 derniers jours.")